In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, make_scorer, recall_score
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from scipy.stats import hmean
import pickle
from sklearn.utils import resample
import xgboost as xgb

In [2]:
Train_Features = [
    "ticker", "company_name", "sector", "industry", "market_cap",
    "price", "open", "close", "high", "low",
    "volume", "adj_close", "dividend", "dividend_yield", "pe_ratio",
    "eps", "beta", "52_week_high", "52_week_low", "shares_outstanding",
    "float", "average_volume", "market", "exchange", "isin",
    "cusip", "country", "currency", "ipo_date", "earnings_date",
    "revenue", "cost_of_goods_sold", "gross_profit", "operating_expenses", "operating_income",
    "ebit", "ebitda", "net_income", "income_before_tax", "tax_expense",
    "net_income_applicable_to_common_shares", "basic_eps", "diluted_eps", "total_assets", "current_assets",
    "non_current_assets", "total_liabilities", "current_liabilities", "non_current_liabilities", "shareholders_equity",
    "retained_earnings", "cash_and_cash_equivalents", "short_term_investments", "long_term_investments", "inventory",
    "accounts_receivable", "accounts_payable", "depreciation", "amortization", "capital_expenditures",
    "loan_id", "loan_amount", "loan_term", "interest_rate", "installment",
    "issue_date", "loan_status", "payment_status", "borrower_score", "borrower_income",
    "debt_to_income", "employment_length", "purpose", "home_ownership", "delinquency_2yrs",
    "credit_score", "fico_range_low", "fico_range_high", "revol_util", "num_open_credit_lines",
    "total_credit_lines", "public_records", "collections_12_mths_ex_med", "application_type", "verification_status",
    "bond_id", "bond_name", "maturity_date", "coupon_rate", "yield_to_maturity",
    "face_value", "issue_price", "current_price", "duration", "convexity",
    "credit_rating", "issuer", "callable", "puttable", "bond_type",
    "transaction_id", "transaction_date", "transaction_amount", "transaction_type", "merchant_name",
    "merchant_category", "account_id", "balance_before", "balance_after", "location",
    "crypto_symbol", "crypto_name", "market_rank", "circulating_supply", "total_supply",
    "max_supply", "market_dominance", "all_time_high", "all_time_low", "last_updated",
    "block_time", "hashing_algorithm", "platform", "explorer_url", "trading_pairs",
    "exchange_rate", "currency_pair", "base_currency", "quote_currency", "rate_date",
    "rate_time", "daily_change", "monthly_change", "yearly_change", "volume_24h",
    "investment_id", "investment_type", "investment_amount", "investment_date", "current_value",
    "gain_loss", "annual_return", "investment_duration", "investment_strategy", "fund_manager",
    "fund_id", "fund_name", "nav", "expense_ratio", "inception_date",
    "fund_category", "assets_under_management", "benchmark_index", "turnover_ratio", "dividend_distribution",
    "gdp", "inflation_rate", "unemployment_rate", "federal_funds_rate", "consumer_price_index",
    "producer_price_index", "retail_sales", "housing_starts", "trade_balance", "government_debt",
    "current_account_balance", "budget_deficit", "foreign_reserves", "money_supply", "taxpayer_id",
    "income_bracket", "taxable_income", "effective_tax_rate", "tax_paid", "deductions",
    "credits", "filing_status", "tax_year", "refund_amount", "bank_id",
    "branch_id", "account_type", "account_open_date", "account_balance", "interest_earned",
    "overdraft_limit", "minimum_balance", "monthly_fee", "account_status", "user_id",
    "customer_id", "registration_date", "last_login", "kyc_status", "risk_score",
    "fraud_flag", "device_id", "ip_address", "login_location", "portfolio_id",
    "asset_class", "allocation_percentage", "benchmark_return", "tracking_error", "sharpe_ratio",
    "alpha", "beta_coefficient", "standard_deviation", "max_drawdown", "audit_status", "accounting_standard", "financial_statement_type", "reporting_currency", "adjustment_reason",
    "deferred_tax_assets", "deferred_tax_liabilities", "intangible_assets", "goodwill", "preferred_equity",
    "policy_id", "policy_holder", "premium_amount", "coverage_amount", "claim_id",
    "claim_status", "underwriting_score", "risk_class", "loss_ratio", "combined_ratio",
    "order_id", "trade_price", "trade_volume", "order_type", "execution_time",
    "bid_price", "ask_price", "spread", "order_book_depth", "trading_halt",
    "swift_code", "iban", "routing_number", "account_opening_method", "branch_location",
    "atm_withdrawals", "wire_transfers", "monthly_statements", "account_tier", "fee_structure",
    "credit_limit", "credit_line_type", "charge_off_status", "days_past_due", "collection_agency",
    "restructuring_status", "forbearance_flag", "loan_purpose", "collateral_type", "repayment_behavior",
    "employment_rate", "labor_force_participation", "consumer_confidence_index", "housing_index", "manufacturing_index",
    "import_volume", "export_volume", "interest_payment", "sovereign_rating", "external_debt",
    "payment_method", "payment_gateway", "settlement_status", "refund_status", "dispute_id",
    "chargeback_amount", "recurring_payment", "subscription_id", "billing_cycle", "invoice_date",
    "wealth_segment", "advisor_id", "fee_schedule", "client_risk_profile", "discretionary_mandate",
    "goals_based_plan", "financial_goal", "investment_objective", "cash_allocation", "equity_allocation",
    "esg_score", "carbon_emission", "sustainability_rating", "board_diversity", "executive_compensation_ratio"
]

In [3]:
Test_Features = [
   "revenue_growth", "net_margin", "operating_margin", "book_value_per_share", "enterprise_value",
   "ev_to_ebitda", "price_to_free_cash_flow", "fcf_margin", "roic", "roa",
   "cash_conversion_cycle", "interest_coverage_ratio", "days_payable_outstanding", "inventory_turnover", "quick_ratio",
   "z_score", "altman_z_score", "short_interest_ratio", "put_call_ratio", "analyst_recommendation",
   "price_target_high", "price_target_low", "estimate_revision", "guidance_change", "buyback_yield",
   "s&p_rating", "moody_rating", "recovery_rate", "default_probability", "credit_spread",
   "real_interest_rate", "velocity_of_money", "consumer_sentiment_index", "labor_cost_index", "construction_spending",
   "crypto_funding_rate", "staking_yield", "token_burn_rate", "dao_votes", "mining_difficulty",
   "digital_wallet_id", "transaction_fee", "payment_token", "subscription_status", "auto_renew_flag",
   "claim_frequency", "premium_to_coverage_ratio", "lapse_rate", "policy_duration", "actuarial_value"
]


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
 # Define preprocessing function
def preprocess_keyword(keyword):
    return keyword.replace("_", " ").lower()


# Load FinancialBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert").to(device)

# Function to generate embeddings using FinancialBERT
def generate_embeddings(keywords, tokenizer, model, device):
    inputs = tokenizer(keywords, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embedding as the sentence representation
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embeddings




In [ ]:

# Preprocess training and testing features
processed_keywords = [preprocess_keyword(k) for k in Train_Features]
processed_testing_keywords = [preprocess_keyword(k) for k in Test_Features]


# Generate embeddings for training and testing data
embeddings = generate_embeddings(processed_keywords, tokenizer, model, device)
testing_embeddings = generate_embeddings(processed_testing_keywords, tokenizer, model, device)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# Perform t-SNE visualization
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
reduced_embeddings = tsne.fit_transform(embeddings)

# Plot t-SNE visualization
plt.figure(figsize=(16, 12))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.7)

# Annotate each point with its corresponding label
for i, label in enumerate(processed_keywords):
    plt.annotate(label, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=7, alpha=0.8)

# Add title and grid
plt.title("t-SNE Visualization of Financial Keyword Embeddings using FinancialBERT", fontsize=14)
plt.grid(True)
plt.show()


NameError: name 'Train_Features' is not defined

In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()
# Initialize the SVM classifier
svm_model = SVC(kernel='rbf', random_state=42, gamma='scale')

threshold_candidates = np.arange(0.8, 0.9, 0.01)

best_threshold = None
best_f1 = 0

min_positives_ratio = 0.01
max_positives_ratio = 0.10


for threshold in threshold_candidates:
    X_temp, y_temp = [], []

    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            vec1, vec2 = embeddings[i], embeddings[j]
            cos_sim = cosine_similarity([vec1], [vec2])[0][0]
            label = 1 if cos_sim > threshold else 0
            X_temp.append(np.concatenate([vec1, vec2]))
            y_temp.append(label)

    X_temp, y_temp = np.array(X_temp), np.array(y_temp)
    positives_ratio = np.mean(y_temp)

    # Skip thresholds with too few or too many positives
    if not (min_positives_ratio <= positives_ratio <= max_positives_ratio):
        print(f"⚠️ Skipping threshold {threshold:.3f} due to positive ratio: {positives_ratio:.4f}")
        continue

    # Split and scale
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train_sub)
    X_val_scaled = scaler.transform(X_val_sub)

    # Train and evaluate
    svm_model.fit(X_train_scaled, y_train_sub)
    y_pred_val = svm_model.predict(X_val_scaled)
    f1 = f1_score(y_val_sub, y_pred_val, zero_division=0)

    print(f"✅ Threshold: {threshold:.3f} | F1: {f1:.4f} | Positive Ratio: {positives_ratio:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

if best_threshold is not None:
    print(f"\n🎯 Best Threshold Found: {best_threshold:.3f} with F1-score: {best_f1:.4f}")
else:
    print("\n❌ No valid threshold found within ratio constraints. Try adjusting the limits.")


⚠️ Skipping threshold 0.800 due to positive ratio: 0.2489
⚠️ Skipping threshold 0.810 due to positive ratio: 0.2062
⚠️ Skipping threshold 0.820 due to positive ratio: 0.1683
⚠️ Skipping threshold 0.830 due to positive ratio: 0.1335
⚠️ Skipping threshold 0.840 due to positive ratio: 0.1016


In [ ]:

# Threshold for compatibility
threshold = 0.85

# Generate all possible pairs of features
X = []  # Input features (concatenated embeddings)
y = []  # Labels (1 for compatible, 0 for incompatible)

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        combined_features = np.concatenate([embeddings[i], embeddings[j]])  # Concatenate embeddings
        X.append(combined_features)
        
        # Compute cosine similarity and assign label
        cos_sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        y.append(1 if cos_sim > threshold else 0)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print(f"Original pairs: {len(X)}")

Original pairs: 43071


In [7]:
def jitter_embedding(embedding, noise_level=0.05):
    noise = np.random.normal(0, noise_level, embedding.shape)
    return embedding + noise

augmented_pairs = []
augmented_labels = []

n_augments = 3  # Number of jittered copies per compatible pair

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        if sim > threshold:
            for _ in range(n_augments):
                vec1_jit = jitter_embedding(embeddings[i])
                vec2_jit = jitter_embedding(embeddings[j])
                augmented_pairs.append(np.concatenate([vec1_jit, vec2_jit]))
                augmented_pairs.append(np.concatenate([embeddings[i], vec2_jit]))
                augmented_pairs.append(np.concatenate([vec1_jit, embeddings[j]]))
                augmented_labels.extend([1, 1, 1])



print(f"Augmented pairs: {len(augmented_pairs)}")
print(f"Original pairs: {len(X)}")
# Combine original and augmented data
X_combined = np.vstack([X, np.array(augmented_pairs)])
y_combined = np.concatenate([y, np.array(augmented_labels)])

Augmented pairs: 29169
Original pairs: 43071


In [8]:
# Threshold for compatibility
threshold = 0.85

# Generate all possible pairs of features
X_Test = []  # Input features (concatenated embeddings)
y_test = []  # Labels (1 for compatible, 0 for incompatible)

for i in range(len(testing_embeddings)):
    for j in range(i + 1, len(testing_embeddings)):
        combined_features = np.concatenate([testing_embeddings[i], testing_embeddings[j]])  # Concatenate embeddings
        X_Test.append(combined_features)
        
        # Compute cosine similarity and assign label
        cos_sim = cosine_similarity([testing_embeddings[i]], [testing_embeddings[j]])[0][0]
        y_test.append(1 if cos_sim > threshold else 0)

# Convert to NumPy arrays
X_Test = np.array(X_Test)
y_test = np.array(y_test)

In [9]:
augmented_pairs = []
augmented_labels = []

n_augments = 3  # Number of jittered copies per compatible pair

for i in range(len(testing_embeddings)):
    for j in range(i + 1, len(testing_embeddings)):
        sim = cosine_similarity([testing_embeddings[i]], [testing_embeddings[j]])[0][0]
        if sim > threshold:
            for _ in range(n_augments):
                vec1_jit = jitter_embedding(testing_embeddings[i])
                vec2_jit = jitter_embedding(testing_embeddings[j])
                augmented_pairs.append(np.concatenate([vec1_jit, vec2_jit]))
                augmented_pairs.append(np.concatenate([testing_embeddings[i], vec2_jit]))
                augmented_pairs.append(np.concatenate([vec1_jit, testing_embeddings[j]]))
                augmented_labels.extend([1, 1, 1])


print(f"Augmented pairs: {len(augmented_pairs)}")
print(f"Original pairs: {len(X_Test)}")
# Combine original and augmented data
X_Test_combined = np.vstack([X_Test, np.array(augmented_pairs)])
y_Test_combined = np.concatenate([y_test, np.array(augmented_labels)])

Augmented pairs: 621
Original pairs: 1225


In [10]:
scaler = StandardScaler()
# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_combined)
X_test_scaled = scaler.transform(X_Test_combined)

In [ ]:
# Define parameter grid for multiple kernels
param_grid = [
    {
        'kernel': ['linear'],
        'C': [0.1, 1, 10]
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.1, 1]
    },
    {
        'kernel': ['poly'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.1, 1],
        'degree': [2, 3, 4]
    }
]

# Initialize the Grid Search
svm = SVC(random_state=42)

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=2,
    scoring='f1',
    verbose=2,
    n_jobs=-1
)

# Run the grid search
grid_search.fit(X_train_scaled, y)

results = grid_search.cv_results_
for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"✔️ Params: {params} | Recall: {mean:.4f} (+/- {std:.4f})")

# Output best model
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


In [58]:
# Define parameter grid for multiple kernels
param_grid = [
    {
        'kernel': ['rbf'],
        'C': list(range(1, 16, 1)),
        'gamma': ['scale', 'auto']
    },
    {
        'kernel': ['poly'],
        'C': list(range(1, 16, 1)),
        'gamma': ['scale', 'auto'],
        'degree': [2, 3]
    }
]

# Initialize the Grid Search
svm = SVC(random_state=42)

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=2,
    scoring='recall',
    verbose=2,
    n_jobs=-1
)

# Run the grid search
grid_search.fit(X_train_scaled, y_combined)

results = grid_search.cv_results_
for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"✔️ Params: {params} | Recall: {mean:.4f} (+/- {std:.4f})")

# Output best model
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


Fitting 2 folds for each of 90 candidates, totalling 180 fits


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[CV] END ........................C=2, gamma=auto, kernel=rbf; total time=59.2min
[CV] END .......................C=2, gamma=scale, kernel=rbf; total time=60.0min
[CV] END ........................C=6, gamma=auto, kernel=rbf; total time=60.3min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=60.4min
[CV] END .......................C=5, gamma=scale, kernel=rbf; total time=60.6min
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=60.7min
[CV] END .......................C=6, gamma=scale, kernel=rbf; total time=60.8min
[CV] END .......................C=4, gamma=scale, kernel=rbf; total time=60.9min
[CV] END ........................C=4, gamma=auto, kernel=rbf; total time=61.1min
[CV] END .......................C=2, gamma=scale, kernel=rbf; total time=71.5min
[CV] END .......................C=3, gamma=scale, kernel=rbf; total time=71.6min
[CV] END ........................C=3, gamma=auto, kernel=rbf; total time=71.7min
[CV] END ...................

In [ ]:
# model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
# model.fit(X_train_scaled, y_combined)

# Predict using best model
y_pred = best_model.predict(X_test_scaled)

# Evaluation
print("\n📊 Classification Report (Test Set):")
print(classification_report(y_Test_combined, y_pred, zero_division=0))

In [ ]:
# 1. Base estimator
xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# 2. Parameter grid around your new “best” run
param_grid = {
    'n_estimators':       [150, 200, 250],
    'learning_rate':      [0.01, 0.03, 0.05],
    'max_depth':          [4, 6, 8],
    'min_child_weight':   [1, 3, 5],
    'subsample':          [0.7, 0.9, 1.0],
    'colsample_bytree':   [0.6, 0.8, 1.0],
    'gamma':              [0.0, 0.1, 0.2],
    'reg_alpha':          [0.0, 0.2, 0.4],
    'reg_lambda':         [0.8, 1.0, 1.2],
    # ratio = (# negative samples) / (# positive samples)
    'scale_pos_weight':   [1, (len(y_combined) - sum(y_combined)) / sum(y_combined)]
}

# Make a scorer that focuses on recall for label=1
recall_pos1 = make_scorer(recall_score, pos_label=1)

# 3. Grid Search (optimize for accuracy; swap scoring to 'recall' if you still want to boost class-1 recall)
grid_search = GridSearchCV(
    estimator   = xgb_base,
    param_grid  = param_grid,
    scoring     = 'recall_pos1',
    cv          = 2,
    n_jobs      = -1,
    verbose     = 1,
    refit       = True
)


# 4. Run the search
grid_search.fit(X_train_scaled, y_combined)

# 5. Best params & CV score
print("🏆 Best parameters found:")
print(grid_search.best_params_)
print(f"Best CV accuracy: {grid_search.best_score_:.4f}\n")

# 6. Test‐set evaluation
y_pred = grid_search.predict(X_test_scaled)
print("📊 Classification Report on TEST:")
print(classification_report(y_Test_combined, y_pred, zero_division=0))


Fitting 2 folds for each of 1024 candidates, totalling 2048 fits


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[CV 2/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.1, reg_lambda=0.8, scale_pos_weight=1, subsample=0.9;, score=(train=0.932, test=0.940) total time= 2.1min
[CV 2/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.2, reg_lambda=0.8, scale_pos_weight=1, subsample=0.8;, score=(train=0.933, test=0.939) total time= 2.1min
[CV 2/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.2, reg_lambda=0.8, scale_pos_weight=1, subsample=0.9;, score=(train=0.932, test=0.941) total time= 2.1min
[CV 2/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.1, reg_lambda=0.8, scale_pos_weight=1, subsample=0.8;, score=(train=0.932, test=0.941) total time= 2.2min
[CV 2/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=

/home/g7/Desktop/Thesis I/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:18:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV 1/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.1, reg_lambda=0.8, scale_pos_weight=1.2289416846652268, subsample=0.8;, score=(train=0.957, test=0.776) total time= 2.2min
[CV 1/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.1, reg_lambda=1.0, scale_pos_weight=1, subsample=0.9;, score=(train=0.939, test=0.709) total time= 2.3min
[CV 1/2] END colsample_bytree=0.6, gamma=0.0, learning_rate=0.03, max_depth=4, min_child_weight=3, n_estimators=150, reg_alpha=0.1, reg_lambda=0.8, scale_pos_weight=1.2289416846652268, subsample=0.9;, score=(train=0.959, test=0.780) total time= 2.3min


...

[CV 1/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=0.8, scale_pos_weight=1.2289416846652268, subsample=0.8;, score=(train=1.000, test=0.879) total time= 3.7min


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/g7/Desktop/Thesis I/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:13:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/g7/Desktop/Thesis I/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:13:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV 2/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=1.0, scale_pos_weight=1, subsample=0.8;, score=(train=0.991, test=0.984) total time= 3.3min
[CV 1/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=1.0, scale_pos_weight=1, subsample=0.8;, score=(train=0.997, test=0.843) total time= 3.5min
[CV 2/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=1.0, scale_pos_weight=1, subsample=0.9;, score=(train=0.990, test=0.982) total time= 3.2min
[CV 1/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=5, n_estimators=200, reg_alpha=0.1, reg_lambda=1.0, scale_pos_weight=1, subsample=0.9;, score=(train=0.998, test=0.844) total time= 3.5min
[CV 2/2] END colsample_bytree=0.8, gamma=0.1, learning_rate=

/home/g7/Desktop/Thesis I/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:15:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🏆 Best parameters:
{'colsample_bytree': 0.8, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5, 'n_estimators': 200, 'reg_alpha': 0.2, 'reg_lambda': 0.8, 'scale_pos_weight': np.float64(1.2289416846652268), 'subsample': 0.8}
Best CV recall (class=1): 0.9353

📋 Top 5 runs by recall:
                                                 params  mean_train_score  \
762   {'colsample_bytree': 0.8, 'gamma': 0.0, 'learn...          0.997346   
1018  {'colsample_bytree': 0.8, 'gamma': 0.1, 'learn...          0.996606   
510   {'colsample_bytree': 0.6, 'gamma': 0.1, 'learn...          0.997562   
466   {'colsample_bytree': 0.6, 'gamma': 0.1, 'learn...          0.997161   
243   {'colsample_bytree': 0.6, 'gamma': 0.0, 'learn...          0.996174   

      mean_test_score  
762          0.935267  
1018         0.934434  
510          0.934280  
466          0.934064  
243          0.933724  

📊 Classification Report on TEST:
              precision    recall  f1-score   suppor

In [27]:
# Convert cv_results_ to DataFrame
cv_results_df = pd.DataFrame(grid_search.cv_results_)

# Sort by validation recall (class 1)
cv_results_df_sorted = cv_results_df.sort_values(by='mean_test_score', ascending=False)

# Select useful columns
display_df = cv_results_df_sorted[[
    'mean_train_score', 'mean_test_score', 'rank_test_score', 'params'
]].rename(columns={
    'mean_train_score': 'Train Recall',
    'mean_test_score': 'Val Recall',
    'rank_test_score': 'Rank'
})

# Format top 10 results
top_results_text = display_df.head(10).to_string(index=False)

# Save to text file
with open("grid_search_top_results.txt", "w") as f:
    f.write("📋 Top 10 Grid Search Results by Validation Recall (class=1):\n\n")
    f.write(top_results_text)

print("✅ Top results saved to 'grid_search_top_results.txt'")


✅ Top results saved to 'grid_search_top_results.txt'


In [ ]:
xgb_model = xgb.XGBClassifier(
    n_estimators=200,             # Slightly reduced to prevent overfitting
    learning_rate=0.05,           # Slower learning for better generalization
    max_depth=6,                  # Shallower trees reduce overfitting
    min_child_weight=5,           # Prevents learning from overly specific patterns
    subsample=0.8,                # Keeps generalization
    colsample_bytree=0.8,         # Same: helps with variance
    gamma=0.0,                    # Adds split regularization
    reg_alpha=0.2,                # L1 regularization
    reg_lambda=0.8,               # L2 regularization
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight = (len(y_combined) - sum(y_combined)) / sum(y_combined)
)

xgb_model.fit(X_train_scaled, y_combined)
y_pred = xgb_model.predict(X_test_scaled)

print("\n📊 XGBoost - Classification Report:")
print(classification_report(y_Test_combined, y_pred, zero_division=0))



📊 XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.87      1156
           1       0.76      0.87      0.81       690

    accuracy                           0.85      1846
   macro avg       0.84      0.85      0.84      1846
weighted avg       0.86      0.85      0.85      1846



In [38]:
with open('model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

In [15]:
xgb_model = pickle.load(open('model.pkl', 'rb'))
scaler = StandardScaler()
# ── Your helper functions ──

def preprocess_keyword(s: str) -> str:
    """
    Normalize a column name for comparison.
    """
    return (
        s.lower()
         .strip()
         .replace("_", " ")
         .replace("-", " ")
         .replace("#", " ")
         .replace("@", " ")
    )

def extract_columns(csv_path: str):
    """
    Read just the header row and return 
    (preprocessed list, raw list) of column names.
    """
    raw = list(pd.read_csv(csv_path, nrows=0).columns)
    pre = [preprocess_keyword(c) for c in raw]
    return pre, raw

def build_subset_mapping(src_pre, tgt_pre):
    """
    Map each src_pre → the longest tgt_pre that strictly contains it.
    """
    mapping = {}
    for s in src_pre:
        matches = [t for t in tgt_pre if s != t and s in t]
        if matches:
            mapping[s] = max(matches, key=len)
    return mapping

# ── Paths & header extraction ──

path1 = "/home/g7/Desktop/Thesis I/Datasets/Ingestor_Datasets/DF_1.csv"
path2 = "/home/g7/Desktop/Thesis I/Datasets/Ingestor_Datasets/DF_2.csv"

cols1_pp, cols1_raw = extract_columns(path1)
cols2_pp, cols2_raw = extract_columns(path2)

# ── Subset‐based renaming ──

map1_pp = build_subset_mapping(cols1_pp, cols2_pp)  # A→B
map2_pp = build_subset_mapping(cols2_pp, cols1_pp)  # B→A

pp_to_raw1 = dict(zip(cols1_pp, cols1_raw))
pp_to_raw2 = dict(zip(cols2_pp, cols2_raw))

rename1 = {
    pp_to_raw1[src]: pp_to_raw2[tgt]
    for src, tgt in map1_pp.items()
    if src in pp_to_raw1 and tgt in pp_to_raw2
}
rename2 = {
    pp_to_raw2[src]: pp_to_raw1[tgt]
    for src, tgt in map2_pp.items()
    if src in pp_to_raw2 and tgt in pp_to_raw1
}

cols1_h = [rename1.get(c, c) for c in cols1_raw]
cols2_h = [rename2.get(c, c) for c in cols2_raw]

print("Harmonized Dataset1 columns:", cols1_h)
print("Harmonized Dataset2 columns:", cols2_h)

# ── Embedding & matching pipeline ──

# 1) Generate embeddings (one arg!)
embeddings_1 = generate_embeddings(cols1_h, tokenizer, model, device)
embeddings_2 = generate_embeddings(cols2_h, tokenizer, model, device)

# 2) Build pairwise embeddings + track pairs
pair_embeddings = []
column_pairs   = []
for i, e1 in enumerate(embeddings_1):
    for j, e2 in enumerate(embeddings_2):
        pair_embeddings.append(np.concatenate([e1, e2]))
        column_pairs.append((cols1_h[i], cols2_h[j]))
pair_embeddings = np.vstack(pair_embeddings)

# 3) Scale — **fit** then transform
pair_scaled = scaler.fit_transform(pair_embeddings)
# If you already have a saved, pre-fitted scaler:
# scaler = load("scaler.joblib")
# pair_scaled = scaler.transform(pair_embeddings)

# 4) Predict
preds = xgb_model.predict(pair_scaled)

# 5) Extract compatible pairs
compatible_pairs = [
    (a, b)
    for (a, b), p in zip(column_pairs, preds)
    if p == 1
]

# 6) Compute coverage & harmonic‐mean score
matched_A = {a for a, _ in compatible_pairs}
matched_B = {b for _, b in compatible_pairs}

coverage_A = len(matched_A) / len(cols1_h)
coverage_B = len(matched_B) / len(cols2_h)
compatibility_score = (hmean([coverage_A, coverage_B])
                       if coverage_A and coverage_B else 0.0)

# 7) Report
print(f"\nCompatibility Score: {compatibility_score:.2f}")
print("Datasets are COMPATIBLE" if compatibility_score >= 0.7 
      else "Datasets are NOT compatible")
print("\n🔗 Compatible Column Pairs:")
if compatible_pairs:
    for a, b in compatible_pairs:
        print(f"- {a} ↔️ {b}")
else:
    print("No compatible columns found.")

output_data = {
    'embeddings_1': embeddings_1,         
    'embeddings_2': embeddings_2,          
    'cols1_h': cols1_h,                    
    'cols2_h': cols2_h,                    
    'compatibility_score': compatibility_score 
}

# Save to a pickle file
with open('compatibility_analysis.pkl', 'wb') as f:
    pickle.dump(output_data, f)

print("\n✅ Embeddings and compatibility data saved to 'compatibility_analysis.pkl'")

Harmonized Dataset1 columns: ['open', 'high', 'low', 'close', 'volume', 'vwap', 'timestamp', 'transactions', 'otc']
Harmonized Dataset2 columns: ['open', 'high', 'low', 'close', 'volume', 'vwap', 'timestamp', 'transactions', 'otc']

Compatibility Score: 0.78
Datasets are COMPATIBLE

🔗 Compatible Column Pairs:
- open ↔️ open
- open ↔️ close
- open ↔️ volume
- open ↔️ vwap
- open ↔️ transactions
- high ↔️ close
- close ↔️ close
- volume ↔️ high
- volume ↔️ close
- volume ↔️ volume
- volume ↔️ transactions
- vwap ↔️ open
- vwap ↔️ volume
- vwap ↔️ vwap
- vwap ↔️ transactions
- timestamp ↔️ vwap
- timestamp ↔️ timestamp
- timestamp ↔️ transactions
- transactions ↔️ open
- transactions ↔️ close
- transactions ↔️ volume
- transactions ↔️ vwap
- transactions ↔️ transactions

✅ Embeddings and compatibility data saved to 'compatibility_analysis.pkl'
